In [1]:
!pip uninstall -y h5py


Found existing installation: h5py 3.11.0
Uninstalling h5py-3.11.0:
  Successfully uninstalled h5py-3.11.0


In [2]:
!pip install h5py

  Obtaining dependency information for h5py from https://files.pythonhosted.org/packages/d8/5e/b7b83cfe60504cc4d24746aed04353af7ea8ec104e597e5ae71b8d0390cb/h5py-3.11.0-cp311-cp311-win_amd64.whl.metadata
  Using cached h5py-3.11.0-cp311-cp311-win_amd64.whl.metadata (2.5 kB)
Using cached h5py-3.11.0-cp311-cp311-win_amd64.whl (3.0 MB)


In [24]:
import cv2
print(cv2.__version__)


4.10.0


In [10]:
!pip install gdown


  Obtaining dependency information for gdown from https://files.pythonhosted.org/packages/54/70/e07c381e6488a77094f04c85c9caf1c8008cdc30778f7019bc52e5285ef0/gdown-5.2.0-py3-none-any.whl.metadata


In [1]:
!pip install opencv-python


In [2]:
import warnings
warnings.filterwarnings("ignore")

from tensorflow.keras.optimizers import Adam, SGD, Nadam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, TensorBoard, LearningRateScheduler
from tensorflow.keras.callbacks import Callback
from tensorflow.keras import backend as K 
from tensorflow.keras.models import load_model
from math import ceil 
import numpy as np 
from termcolor import colored
import h5py
from mn_model import mn_model
from face_generator import BatchGenerator
from keras_ssd_loss import SSDLoss
from ssd_box_encode_decode_utils import SSDBoxEncoder, decode_y, decode_y2

import scipy.misc as sm
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0' # choose gpu

In [3]:
img_height = 300
img_width = 300
img_channels = 3

n_classes = 2 
class_names = ["background","face"]

scales = [0.05, 0.10, 0.15, 0.20, 0.30, 0.45, 0.60]
aspect_ratios = [[0.5, 1.0, 2.0],
                 [1.0/3.0, 0.5, 1.0, 2.0, 3.0],
                 [1.0/3.0, 0.5, 1.0, 2.0, 3.0],
                 [1.0/3.0, 0.5, 1.0, 2.0, 3.0],
                 [0.5, 1.0, 2.0],
                 [0.5, 1.0, 2.0]]
      
     # The anchor box aspect ratios used in the original SSD300
two_boxes_for_ar1 = True
limit_boxes = True # Whether or not you want to limit the anchor boxes to lie entirely within the image boundaries
variances = [0.1, 0.1, 0.2, 0.2] # The variances by which the encoded target coordinates are scaled as in the original implementation
coords = 'centroids' # Whether the box coordinates to be used as targets for the model should be in the 'centroids' or 'minmax' format, see documentation
normalize_coords = True

data_path = 'dataset/'
det_model_path = "./models/" 
train_data = data_path + 'wider_train_v1.npy'
test_data = data_path + 'wider_val_v1.npy'


In [3]:
import zipfile
import os

# Define the path to the ZIP file and the extraction directory
zip_file_path = 'C:/Users/volet/Downloads/WIDER_train.zip'  # Replace with the path to your ZIP file
extract_dir = './dataset'      # Replace with the path where you want to extract the files

# Create the extraction directory if it doesn't exist
os.makedirs(extract_dir, exist_ok=True)

# Extract the ZIP file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print(f'Files extracted to {extract_dir}')


KeyboardInterrupt: 

In [4]:
zip_file_path = 'C:/Users/volet/Downloads/WIDER_val.zip'  # Replace with the path to your ZIP file
extract_dir = './dataset'      # Replace with the path where you want to extract the files

# Create the extraction directory if it doesn't exist
os.makedirs(extract_dir, exist_ok=True)

# Extract the ZIP file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print(f'Files extracted to {extract_dir}')


Files extracted to ./dataset


In [4]:
# build the keras model
# this model is not retrained, we are doing it from scratch 

model, model_layer, img_input, predictor_sizes = mn_model(
    image_size=(img_height, img_width, img_channels), 
    n_classes=n_classes,
    min_scale=None, 
    max_scale=None, 
    scales=scales, 
    aspect_ratios_global=None, 
    aspect_ratios_per_layer=aspect_ratios, 
    two_boxes_for_ar1=two_boxes_for_ar1, 
    limit_boxes=limit_boxes, 
    variances=variances, 
    coords=coords, 
    normalize_coords=normalize_coords
)

#model.summary()

print ("Freezing classification layers")
#Freeze layers
for layer_key in model_layer:
    if('detection'  not in layer_key): #prefix detection to freeze layers which does not have detection
        model_layer[layer_key].trainable = False
print (colored("classification layers freezed", 'green'))

# for layer in model.layers:
#   print (colored(layer.name, 'blue'))
#   print (colored(layer.trainable, 'green'))

print ("loading classification weights")
classification_model = './base_models/mobilenet_1_0_224_tf.h5'
model.load_weights(classification_model,  by_name= True)

print (colored( ('classification weights %s loaded' % classification_model), 'green'))


Freezing classification layers
classification layers freezed
loading classification weights
classification weights ./base_models/mobilenet_1_0_224_tf.h5 loaded


In [5]:
# setting up taining 

batch_size = 256
num_epochs = 25

#Adam
base_lr = 0.001
adam = Adam(learning_rate=base_lr, beta_1=0.9, beta_2=0.999, epsilon=1e-6, decay = 0.0)
ssd_loss = SSDLoss(neg_pos_ratio=2, n_neg_min=0, alpha=1.0, beta = 1.0)
model.compile(optimizer=adam, loss=ssd_loss.compute_loss)


ssd_box_encoder = SSDBoxEncoder(img_height=img_height,
                                img_width=img_width,
                                n_classes=n_classes, 
                                predictor_sizes=predictor_sizes,
                                min_scale=None,
                                max_scale=None,
                                scales=scales,
                                aspect_ratios_global=None,
                                aspect_ratios_per_layer=aspect_ratios,
                                two_boxes_for_ar1=two_boxes_for_ar1,
                                limit_boxes=limit_boxes,
                                variances=variances,
                                pos_iou_threshold=0.6,
                                neg_iou_threshold=0.4,
                                coords=coords,
                                normalize_coords=normalize_coords)

train_dataset = BatchGenerator(images_path=train_data, 
                               include_classes='all', 
                               box_output_format = ['class_id', 'xmin', 'xmax', 'ymin', 'ymax'])

print ("TRAINING DATA")

train_dataset.parse_xml(
                  annotations_path=train_data,
                  image_set_path=data_path,
                  image_set='None',
                  classes = class_names, 
                  exclude_truncated=False,
                  exclude_difficult=False,
                  ret=False, 
                  debug = False)

train_generator = train_dataset.generate(
                 batch_size=batch_size,
                 train=True,
                 ssd_box_encoder=ssd_box_encoder,
                 equalize=True,
                 brightness=(0.5,2,0.5),
                 flip=0.5,
                 translate=((0, 20), (0, 30), 0.5),
                 scale=(0.75, 1.2, 0.5),
                 crop=False,
                 #random_crop = (img_height,img_width,1,3), 
                 random_crop=False,
                 resize=(img_height, img_width),
                 #resize=False,
                 gray=False,
                 limit_boxes=True,
                 include_thresh=0.4,
                 diagnostics=False)

n_train_samples = train_dataset.get_n_samples()

print ("Total number of training samples = {}".format(n_train_samples))


print ("VALIDATION DATA")

val_dataset = BatchGenerator(images_path=test_data, include_classes='all', 
                box_output_format = ['class_id', 'xmin', 'xmax', 'ymin', 'ymax'])


val_dataset.parse_xml(
                  annotations_path=test_data,
                  image_set_path=data_path,
                  image_set='None',
                  classes = class_names, 
                  exclude_truncated=False,
                  exclude_difficult=False,
                  ret=False, 
                  debug = False)


val_generator = val_dataset.generate(
                 batch_size=batch_size,
                 train=True,
                 ssd_box_encoder=ssd_box_encoder,
                 equalize=False,
                 brightness=False,
                 flip=False,
                 translate=False,
                 scale=False,
                 crop=False,
                 #random_crop = (img_height,img_width,1,3), 
                 random_crop=False, 
                 resize=(img_height, img_width), 
                 #resize=False, 
                 gray=False,
                 limit_boxes=True,
                 include_thresh=0.4,
                 diagnostics=False)

n_val_samples = val_dataset.get_n_samples()

print ("Total number of validation samples = {}".format(n_val_samples))

TRAINING DATA
Total number of training samples = 12619
VALIDATION DATA
Total number of validation samples = 3143


In [6]:
model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 300, 300,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ preprocess (Lambda) │ (None, 300, 300,  │          0 │ input_layer[0][0] │
│                     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1 (Conv2D)      │ (None, 150, 150,  │        864 │ preprocess[0][0]  │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, 150, 150,  │        128 │ conv1[0][0]       │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, 150, 150,  │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv_dw_1           │ (None, 150, 150,  │        288 │ conv1_relu[0][0]  │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv_dw_1_bn        │ (None, 150, 150,  │        128 │ conv_dw_1[0][0]   │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv_dw_1_relu      │ (None, 150, 150,  │          0 │ conv_dw_1_bn[0][… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv_pw_1 (Conv2D)  │ (None, 150, 150,  │      2,048 │ conv_dw_1_relu[0… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv_pw_1_bn        │ (None, 150, 150,  │        256 │ conv_pw_1[0][0]   │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv_pw_1_relu      │ (None, 150, 150,  │          0 │ conv_pw_1_bn[0][… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv_dw_2           │ (None, 75, 75,    │        576 │ conv_pw_1_relu[0… │
│ (DepthwiseConv2D)   │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv_dw_2_bn        │ (None, 75, 75,    │        256 │ conv_dw_2[0][0]   │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv_dw_2_relu      │ (None, 75, 75,    │          0 │ conv_dw_2_bn[0][… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv_pw_2 (Conv2D)  │ (None, 75, 75,    │      8,192 │ conv_dw_2_relu[0… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv_pw_2_bn        │ (None, 75, 75,    │        512 │ conv_pw_2[0][0]   │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv_pw_2_relu      │ (None, 75, 75,    │          0 │ conv_pw_2_bn[0][

 Total params: 4,634,356 (17.68 MB)

 Trainable params: 377,920 (1.44 MB)

 Non-trainable params: 4,256,436 (16.24 MB)

In [7]:
from tensorflow.keras.callbacks import LearningRateScheduler, ReduceLROnPlateau, TensorBoard, EarlyStopping, ModelCheckpoint
from math import ceil
import tensorflow.keras.backend as K

# Ensure the model is properly defined before this code block
# model = ... (Your model definition here)

# Learning rate scheduler function
def scheduler(epoch, lr):
    if epoch % 10 == 0 and epoch != 0:
        new_lr = lr * 0.95
        print(f"Learning rate changed to {new_lr}")
        return new_lr
    else:
        print(f"Learning rate remains {lr}")
        return lr

# Callbacks
lr_schedule = LearningRateScheduler(scheduler)
plateau = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.3,
    patience=6,
    min_delta=0.001,  # 'epsilon' argument was renamed to 'min_delta'
    cooldown=2
)
tensorboard = TensorBoard(
    log_dir='./logs/trial1/',
    histogram_freq=1,
    write_graph=True,
    write_images=True,
    embeddings_freq=0
)
early_stopping = EarlyStopping(
    monitor='val_loss',
    min_delta=0.001,
    patience=100  # Adjusted to a more reasonable number for early stopping
)
model_checkpoint = ModelCheckpoint(
    filepath=det_model_path + 'ssd_mobilenet_face_epoch_{epoch:02d}_loss{val_loss:.4f}.weights.h5',
    monitor='val_loss',
    verbose=1,
    save_best_only=True,
    save_weights_only=True,
    mode='auto'
)

# Fit the model
history = model.fit(
    x=train_generator,
    steps_per_epoch=ceil(n_train_samples / batch_size),
    epochs=num_epochs,
    callbacks=[model_checkpoint, lr_schedule, early_stopping, plateau, tensorboard],  # Added TensorBoard callback
    validation_data=val_generator,
    validation_steps=ceil(n_val_samples / batch_size)
)

# Save weights
model.save_weights(det_model_path + f'ssd_mobilenet_weights_epoch_{num_epochs}.weights.h5')

print(f"Model and weight files saved at: {det_model_path}")


Learning rate remains 0.0010000000474974513
Epoch 1/25
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 16s/step - loss: 0.0679 
Epoch 1: val_loss improved from inf to 0.07133, saving model to ./models/ssd_mobilenet_face_epoch_01_loss0.0713.weights.h5
50/50 ━━━━━━━━━━━━━━━━━━━━ 978s 20s/step - loss: 0.0679 - val_loss: 0.0713 - learning_rate: 0.0010
Learning rate remains 0.0010000000474974513
Epoch 2/25
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 17s/step - loss: 0.0674 
Epoch 2: val_loss improved from 0.07133 to 0.07108, saving model to ./models/ssd_mobilenet_face_epoch_02_loss0.0711.weights.h5
50/50 ━━━━━━━━━━━━━━━━━━━━ 1022s 21s/step - loss: 0.0674 - val_loss: 0.0711 - learning_rate: 0.0010
Learning rate remains 0.0010000000474974513
Epoch 3/25
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 17s/step - loss: 0.0668 
Epoch 3: val_loss improved from 0.07108 to 0.07089, saving model to ./models/ssd_mobilenet_face_epoch_03_loss0.0709.weights.h5
50/50 ━━━━━━━━━━━━━━━━━━━━ 1018s 21s/step - loss: 0.0668 - val_loss: 0.0709 - learning_rate: 0

Learning rate remains 2.4367500373045914e-05
Epoch 24/25
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 16s/step - loss: 0.0669 
Epoch 24: val_loss did not improve from 0.07026
50/50 ━━━━━━━━━━━━━━━━━━━━ 925s 19s/step - loss: 0.0670 - val_loss: 0.0703 - learning_rate: 2.4368e-05
Learning rate remains 2.4367500373045914e-05
Epoch 25/25
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 15s/step - loss: 0.0669 
Epoch 25: val_loss did not improve from 0.07026
50/50 ━━━━━━━━━━━━━━━━━━━━ 863s 17s/step - loss: 0.0669 - val_loss: 0.0705 - learning_rate: 2.4368e-05
Model and weight files saved at: ./models/


In [8]:
model_path = './models/'
model_name = 'ssd_mobilenet_weights_epoch_25.weights.h5'
model.load_weights(model_path + model_name)
print(f'weights {model_path + model_name} loaded')

weights ./models/ssd_mobilenet_weights_epoch_25.weights.h5 loaded


In [11]:
from keras.preprocessing import image
from matplotlib import pyplot as plt
from face_generator import save_bb
test_size = 16
test_generator = val_dataset.generate(
    batch_size=test_size,
    train=False,
    ssd_box_encoder=ssd_box_encoder,
    equalize=False,
    brightness=False,
    flip=False,
    translate=False,
    scale=False,
    crop=False,
    random_crop=False,
    resize=(img_height, img_width),
    gray=False,
    limit_boxes=True,
    include_thresh=0.2,
    diagnostics=False
)

print(colored("done.", "green"))
print(colored("now predicting...", "yellow"))

_CONF = 0.7 
_IOU = 0.15

# Get a batch of images, ground truth labels, and filenames
X, y, filenames = next(test_generator)

# Predict on the batch
y_pred = model.predict(X)

# Decode the predictions
y_pred_decoded = decode_y2(y_pred,
                           confidence_thresh=_CONF,
                           iou_threshold=_IOU,
                           top_k="all",
                           input_coords=coords,
                           normalize_coords=normalize_coords,
                           img_height=img_height,
                           img_width=img_width)

np.set_printoptions(suppress=True)

# Ensure filenames and y_pred_decoded have the same length
num_files = len(filenames)
num_preds = len(y_pred_decoded)

if num_files != num_preds:
    raise ValueError(f"Mismatch in number of predictions ({num_preds}) and files ({num_files}).")

# Iterate over the batch
for i in range(num_files):
    save_bb("./output_test/", filenames[i], y_pred_decoded[i])
    save_bb("./output_test/", filenames[i], y[i], prediction=False)

print(colored("Prediction and saving complete.", "green"))


done.
now predicting...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 438ms/step
y_pred shape: (16, 2268, 6)
Total number of bounding boxes: 180
Saved: ./output_test/37_Soccer_Soccer_37_114.jpg
Total number of bounding boxes: 2
Saved: ./output_test/37_Soccer_Soccer_37_114_gt.jpg
Total number of bounding boxes: 176
Saved: ./output_test/49_Greeting_peoplegreeting_49_192.jpg
Total number of bounding boxes: 3
Saved: ./output_test/49_Greeting_peoplegreeting_49_192_gt.jpg
Total number of bounding boxes: 245
Saved: ./output_test/28_Sports_Fan_Sports_Fan_28_663.jpg
Total number of bounding boxes: 1
Saved: ./output_test/28_Sports_Fan_Sports_Fan_28_663_gt.jpg
Total number of bounding boxes: 235
Saved: ./output_test/47_Matador_Bullfighter_matadorbullfighting_47_152.jpg
Total number of bounding boxes: 9
Saved: ./output_test/47_Matador_Bullfighter_matadorbullfighting_47_152_gt.jpg
Total number of bounding boxes: 206
Saved: ./output_test/16_Award_Ceremony_Awards_Ceremony_16_59.jpg
Total number of bounding boxes: 1
Sa